In [1]:
#  Forked from rough_work_2
#  this time increasing n-est to 5k with same 20skf on every t100
#  don't forget to changge output file names....

In [2]:
#  add is_sam feature (will it be useful.. or confuse the model ??)
#  the mean_std-df is redundant

In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
np.random.SeedSequence(RANDOM_SEED)
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [4]:
metadata = pd.read_csv('../input/is-this-crazy/metadata.csv')
sup_metadata = pd.read_csv('../input/is-this-crazy/supplemental_metadata.csv')
train_labels = pd.read_csv('../input/is-this-crazy/train_labels.csv', index_col="sample_id")
val_labels = pd.read_csv('../input/is-this-crazy/val_labels.csv',index_col='sample_id')
# targets = pd.read_csv('targets.csv',index_col='sample_id')
submission_format = pd.read_csv('../input/is-this-crazy/submission_format.csv')

In [5]:
targets = pd.concat([train_labels,val_labels],axis=0)
targets.shape

(1059, 10)

In [6]:
sam_id = metadata[metadata.instrument_type == 'sam_testbed'].index

In [7]:
%%time
test_100 = pd.read_csv('../input/train-rg-transformed/test_100.csv',index_col=0,header=[0,1])
train_100 = pd.read_csv('../input/train-rg-transformed/train_100.csv',index_col=0,header=[0,1])
val_100 = pd.read_csv('../input/train-rg-transformed/val_100.csv',index_col=0,header=[0,1])

val_100.index = metadata[metadata.split == 'val'].sample_id

val_100.sort_index(inplace=True,axis=1)
test_100.sort_index(inplace=True,axis=1)
train_100.sort_index(inplace=True,axis=1)

CPU times: user 769 ms, sys: 97 ms, total: 866 ms
Wall time: 1.12 s


In [8]:
# train_100.drop(columns=[('sample_id_encoded', 'Unnamed: 1601_level_1')],axis=1,inplace=True)

In [9]:
train_set_200 = pd.concat([train_100,val_100],axis=0)
test_set_200 = pd.concat([val_100,test_100],axis=0)
train_set_200.shape, test_set_200.shape

((1059, 2100), (804, 2100))

In [10]:
%%time
train_50 = pd.read_csv('../input/train-rg-transformed/train_50.csv',index_col=0,header=[0,1])
test_50 = pd.read_csv('../input/train-rg-transformed/test_50.csv',index_col=0,header=[0,1])
val_50 = pd.read_csv('../input/train-rg-transformed/val_50.csv',index_col=0,header=[0,1])

test_50.sort_index(inplace=True,axis=1)
train_50.sort_index(inplace=True,axis=1)
val_50.sort_index(inplace=True,axis=1)

CPU times: user 1.76 s, sys: 146 ms, total: 1.91 s
Wall time: 2.38 s


In [11]:
train_set_50 = pd.concat([train_50,val_50],axis=0)
test_set_50 = pd.concat([val_50,test_50],axis=0)
train_set_50.shape, test_set_50.shape

((1059, 4200), (804, 4200))

In [12]:
train_200 = pd.read_csv('../input/train-rg-transformed/train_200.csv',index_col=0,header=[0,1])
val_200 = pd.read_csv('../input/train-rg-transformed/val_200.csv',index_col=0,header=[0,1])
test_200 = pd.read_csv('../input/train-rg-transformed/test_200.csv',index_col=0,header=[0,1])

test_200.sort_index(inplace=True,axis=1)
val_200.sort_index(inplace=True,axis=1)
train_200.sort_index(inplace=True,axis=1)

In [13]:
train_set_200 = pd.concat([train_200,val_200],axis=0)
test_set_200 = pd.concat([val_200,test_200],axis=0)
train_set_200.shape, test_set_200.shape

((1059, 1000), (804, 1000))

In [14]:
skf = StratifiedKFold(n_splits=20,random_state=RANDOM_SEED,shuffle=True)
log_loss_scorer = make_scorer(log_loss,needs_proba=True)

In [15]:
phy = 'phyllosilicate'
bas = 'basalt'
lr = LogisticRegression(solver='liblinear',C=10,random_state=RANDOM_SEED,penalty='l1')
lr

LogisticRegression(C=10, penalty='l1', random_state=777, solver='liblinear')

In [16]:
def get_coefs(X,y,c):
    lr = LogisticRegression(solver='liblinear',C=c,random_state=RANDOM_SEED,penalty='l1')
    print(cross_val_score(lr,X,y,cv=skf,scoring=log_loss_scorer).mean())
    lr.fit(X,y)
    # print(log_loss(y,lr.predict_proba(X)))
    coefs =  lr.coef_
    coefs_list  = []
    for id,wt in enumerate(tqdm(coefs[0])):
        if wt != 0:
            coefs_list.append(id)
    print(len(coefs_list))
    return coefs_list

In [17]:
def get_mz(mz,df):
    ''' returns list of columns for given mz'''
    cols_list = []
    for col in df.columns:
        if col[0] == str(mz):
            cols_list.append(col)
    return cols_list

In [18]:
def get_total_abundance(df):
    ''' returns total abundance for each sample'''
    tot_abund_df = pd.DataFrame(index=df.index)
    tot_abund_df['tot_abund'] = df.sum(axis=1)
    for mz in tqdm(range(0,100)):
        tot_abund_df[f"tot_abund_{mz}"] = df[get_mz(mz,df)].sum(axis=1)
    print(tot_abund_df.shape)
    return tot_abund_df

In [19]:
def get_grads(df,bin_size=5):
    ''' returns gradients for each mz'''
    grad_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        # df_mz_col1 = df_mz.iloc[:,0].values
        df_mz = df_mz.diff(axis=1)
        # df_mz.iloc[:,0] = df_mz_col1
        df_mz.dropna(axis='columns',inplace=True)
        # df_mz = df_mz/bin_size
        df_mz.columns = [col[0]+ '_' + col[1] + '_' + 'grad' for col in df_mz.columns]
        grad_df = pd.concat([grad_df,df_mz],axis=1)
    print(grad_df.shape)

    return  grad_df

In [20]:
def get_mean_std(df):
    ''' returns mean and std for each mz'''
    mean_std_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        df_mz[f'{mz}_mean'] = df_mz.mean(axis=1)
        df_mz[f'{mz}_std'] = df_mz.std(axis=1)
        mean_std_df = pd.concat([mean_std_df,df_mz],axis=1)
    print(mean_std_df.shape)
    return mean_std_df

In [21]:
# def add_fts(df):
#     tot_abund_df = get_total_abundance(df)
#     grad_df = get_grads(df)
#     mean_std_df = get_mean_std(df)
#     new_df = pd.concat([tot_abund_df,grad_df,mean_std_df],axis=1)
#     return new_df

In [22]:
train_set_200.shape, test_set_200.shape

((1059, 1000), (804, 1000))

In [23]:
assert (train_set_200.columns == test_set_200.columns).all()

In [24]:
t_100 = get_total_abundance(train_set_200)
g_100 = get_grads(train_set_200)
m_100 = get_mean_std(train_set_200)

100%|██████████| 100/100 [00:00<00:00, 276.56it/s]


(1059, 101)


100%|██████████| 100/100 [00:00<00:00, 112.46it/s]


(1059, 900)


100%|██████████| 100/100 [00:00<00:00, 128.01it/s]

(1059, 1200)


In [25]:
t_100_test = get_total_abundance(test_set_200)
g_100_test = get_grads(test_set_200)
m_100_test = get_mean_std(test_set_200)


100%|██████████| 100/100 [00:00<00:00, 255.89it/s]


(804, 101)


100%|██████████| 100/100 [00:00<00:00, 106.42it/s]


(804, 900)


100%|██████████| 100/100 [00:00<00:00, 120.54it/s]

(804, 1200)


In [26]:
t_100_test.sort_index(inplace=True,axis=1)
t_100.sort_index(inplace=True,axis=1)

m_100_test.sort_index(inplace=True,axis=1)
m_100.sort_index(inplace=True,axis=1)

g_100_test.sort_index(inplace=True,axis=1)
g_100.sort_index(inplace=True,axis=1)

In [27]:
assert (t_100.columns == t_100_test.columns).all()
assert (g_100.columns == g_100_test.columns).all()
assert (m_100.columns == m_100_test.columns).all()

In [28]:
ntrain = 1059
ntest = 804
NFOLDS = 20

In [29]:
oof_train_df = pd.DataFrame(index=metadata[metadata.split != 'test'].index, columns=['sample_id',*targets.columns])
oof_test_df = pd.DataFrame(index=metadata[metadata.split != 'train'].index, columns=['sample_id',*targets.columns])

In [30]:
agg_loss = []
for target in targets.columns:
    print('TARGET ',target)
    imp_fts = {}
    
    #  get all imp coefs for the target
    t_100_coefs = get_coefs(t_100,targets[target],1)
    g_100_coefs = get_coefs(g_100,targets[target],5)
    m_100_coefs = get_coefs(m_100,targets[target],10)
    o_100_coefs = get_coefs(train_set_200,targets[target],10)
    
    imp_fts['o_100_coefs'] = o_100_coefs
    imp_fts['t_100_coefs'] = t_100_coefs
    imp_fts['g_100_coefs'] = g_100_coefs
    imp_fts['m_100_coefs'] = m_100_coefs

    # get all imp_dfs and concat them
    train_imp_ = train_set_200.iloc[:,o_100_coefs]
    t_100_imp_ = t_100.iloc[:,t_100_coefs]
    g_100_imp_ = g_100.iloc[:,g_100_coefs]
    m_100_imp_ = m_100.iloc[:,m_100_coefs]

    print(train_imp_.shape,t_100_imp_.shape,g_100_imp_.shape,m_100_imp_.shape)
    X = pd.concat([train_imp_,t_100_imp_,g_100_imp_,m_100_imp_],axis=1)
    print(X.shape)

    # same goes for test set
    test_imp_ = test_set_200.iloc[:,o_100_coefs]
    t_100_imp_test = t_100_test.iloc[:,t_100_coefs]
    g_100_imp_test = g_100_test.iloc[:,g_100_coefs]
    m_100_imp_test = m_100_test.iloc[:,m_100_coefs]

    print(test_imp_.shape,t_100_imp_test.shape,g_100_imp_test.shape,m_100_imp_test.shape)
    
    X_test = pd.concat([test_imp_, t_100_imp_test, g_100_imp_test, m_100_imp_test],axis=1)
    print(X_test.shape)
    
    #  add feature
    X['is_sam'] = 0
    X.loc[sam_id[:12],'is_sam'] = 1
    X_test['is_sam'] = 0
    X_test.loc[sam_id[12:],'is_sam'] = 1
    
    
    assert (X.columns == X_test.columns).all()

    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    Y = targets[target]

    cb_clf = CatBoostClassifier(verbose=999,random_seed=RANDOM_SEED,n_estimators=5000)
    cv_loss = []
    for fold, (train_id, test_id) in enumerate(tqdm(skf.split(X,Y))):

        X_train, Y_train = X.iloc[train_id,], Y.iloc[train_id,]
        X_val, Y_val = X.iloc[test_id,], Y.iloc[test_id,]
        #  ohh man unwanted semi colon in iloc throws an error...
        
        # train model  and pred on oof folds
        cb_clf.fit(X_train.values,Y_train)
        y_preds = cb_clf.predict_proba(X_val.values)[:,1]
        fold_loss = log_loss(Y_val,y_preds)
        print(f'fold Loss {fold}: ',fold_loss)
        cv_loss.append(fold_loss)

        oof_train[test_id] = y_preds
        oof_test_skf[fold,:] = cb_clf.predict_proba(X_test.values)[:,1]
        
        print("---------------------------------------------------------------------------")
        
    print('Training Loss:', log_loss(Y,cb_clf.predict_proba(X.values)[:,1]))
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test_df[target] = oof_test
    oof_train_df[target] = oof_train

    print(f'Loss for {target}:')
    print(np.array(cv_loss).mean(),np.array(cv_loss).std())
    agg_loss.append(np.array(cv_loss).mean())
    print("==================================================================================================")

print('AGG Loss:',np.array(agg_loss))
print('AGG Loss mean:',np.array(agg_loss).mean())

TARGET  basalt
0.3178355720052564


100%|██████████| 101/101 [00:00<00:00, 208476.72it/s]

10


0.28539622052357394


100%|██████████| 900/900 [00:00<00:00, 1331055.57it/s]

...

4995:	learn: 0.0046272	total: 1m 7s	remaining: 53.9ms
4999:	learn: 0.0046209	total: 1m 7s	remaining: 0us
fold Loss 17:  0.08607570958022942
---------------------------------------------------------------------------
Learning rate set to 0.002361
0:	learn: 0.6878804	total: 19.6ms	remaining: 1m 37s
999:	learn: 0.0373258	total: 13.6s	remaining: 54.3s
1998:	learn: 0.0153432	total: 27s	remaining: 40.6s
2997:	learn: 0.0087306	total: 40.3s	remaining: 26.9s
3996:	learn: 0.0057310	total: 53.8s	remaining: 13.5s


19it [21:41, 68.27s/it]

4995:	learn: 0.0042424	total: 1m 7s	remaining: 53.8ms
4999:	learn: 0.0042378	total: 1m 7s	remaining: 0us
fold Loss 18:  0.0657393646747153
---------------------------------------------------------------------------
Learning rate set to 0.002362
0:	learn: 0.6888205	total: 19.6ms	remaining: 1m 37s
999:	learn: 0.0356505	total: 13.7s	remaining: 54.7s
1998:	learn: 0.0142865	total: 27s	remaining: 40.5s
2997:	learn: 0.0081004	total: 40.5s	remaining: 27.1s
3996:	learn: 0.0053895	total: 53.9s	remaining: 13.5s


20it [22:49, 68.46s/it]

4995:	learn: 0.0038481	total: 1m 7s	remaining: 53.8ms
4999:	learn: 0.0038447	total: 1m 7s	remaining: 0us
fold Loss 19:  0.12336370562098899
---------------------------------------------------------------------------
Training Loss: 0.009718581163399218
Loss for sulfide:
0.06304194554652578 0.05041156241580525
AGG Loss: [0.12833769 0.07830791 0.15321949 0.19756311 0.00575638 0.14176394
 0.21407855 0.1712562  0.16161721 0.06304195]
AGG Loss mean: 0.1314942416744978


In [31]:
print(np.array(agg_loss).mean())

0.1314942416744978


In [32]:
test_preds = oof_test_df.iloc[:293]
train_preds = oof_train_df.iloc[766:]

In [33]:
display(test_preds)
display(train_preds)

,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.002205,0.001679,0.010964,0.007103,0.000552,0.004424,0.012242,0.091159,0.008306,0.001484
767,NaN,0.030568,0.006904,0.002112,0.014314,0.000055,0.012394,0.010269,0.140084,0.021893,0.000770
768,NaN,0.929668,0.106316,0.001258,0.101595,0.000435,0.020045,0.817536,0.889520,0.027732,0.000870
769,NaN,0.008095,0.003535,0.032972,0.032851,0.000041,0.967057,0.044719,0.005771,0.962424,0.000585
770,NaN,0.005059,0.003026,0.026366,0.677561,0.000075,0.983361,0.972816,0.003374,0.003930,0.000303
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.004921,0.003283,0.011794,0.016838,0.000150,0.006136,0.013741,0.033849,0.009203,0.000293
1055,NaN,0.007255,0.004034,0.011545,0.032297,0.002404,0.014885,0.012135,0.856998,0.002497,0.000631
1056,NaN,0.002952,0.005595,0.001866,0.011255,0.998571,0.002108,0.003464,0.003171,0.001589,0.001072
1057,NaN,0.004113,0.947583,0.013057,0.027860,0.000669,0.004244,0.035782,0.007563,0.003277,0.000864


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.002398,0.001797,0.016330,0.007253,0.000612,0.004481,0.014679,0.484733,0.009494,0.001579
767,NaN,0.075871,0.001942,0.002133,0.021583,0.000067,0.019317,0.017426,0.596806,0.031315,0.000734
768,NaN,0.856404,0.857426,0.001407,0.249915,0.000417,0.030787,0.453863,0.664901,0.082379,0.000843
769,NaN,0.010354,0.004412,0.071149,0.042800,0.000056,0.948198,0.068691,0.006324,0.903269,0.000406
770,NaN,0.003816,0.003446,0.034373,0.112772,0.000083,0.979115,0.971893,0.004256,0.004456,0.000369
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.004037,0.002161,0.010346,0.020913,0.000152,0.005660,0.019861,0.069877,0.016827,0.000253
1055,NaN,0.011191,0.003987,0.012643,0.045951,0.003435,0.023496,0.016006,0.182455,0.002378,0.000646
1056,NaN,0.003605,0.008111,0.001905,0.012997,0.998504,0.002137,0.003671,0.003171,0.001614,0.001342
1057,NaN,0.006781,0.899968,0.013264,0.048984,0.000588,0.006311,0.052713,0.009762,0.004168,0.001135


In [34]:
agg_loss_train = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],train_preds[tar])
    print(tar, loss)
    agg_loss_train.append(loss)
print(np.array(agg_loss_train).mean())

basalt 0.13241687083030249
carbonate 0.06471385691487794
chloride 0.16479516301717553
iron_oxide 0.25095641131877455
oxalate 0.0013969475995482233
oxychlorine 0.13360848014369486
phyllosilicate 0.20225190288945807
silicate 0.14984780274336812
sulfate 0.18878138067313668
sulfide 0.06812499507970128
0.13568938112100376


In [35]:
agg_loss_test = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],test_preds[tar])
    print(tar, loss)
    agg_loss_test.append(loss)
print(np.array(agg_loss_test).mean())

basalt 0.024952697193513663
carbonate 0.012466407743214574
chloride 0.03744553697080144
iron_oxide 0.05749700001667488
oxalate 0.0005754830976788787
oxychlorine 0.027054368751779845
phyllosilicate 0.052611319762747064
silicate 0.03674604053785433
sulfate 0.03209238022986162
sulfide 0.005889940461604966
0.028733117476573124


In [36]:
oof_test_df.sample_id = metadata[metadata.split != 'train'].sample_id
oof_train_df.sample_id = metadata[metadata.split != 'test'].sample_id

In [37]:
oof_train_df.to_csv('oof_train_100_20fold_5kest.csv', index=False)
oof_test_df.to_csv('oof_test_100_20fold_5kest.csv', index=False)